In [1]:
# %load_ext autoreload
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys
import time
import pdb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import networkx as nx

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import f1_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.model_selection import train_test_split

# Clustering
from sklearn.datasets import make_biclusters, make_checkerboard, make_classification
from sklearn.datasets import samples_generator as sg
from sklearn.cluster.bicluster import SpectralCoclustering, SpectralBiclustering
from sklearn.metrics import consensus_score

# Corr
from scipy.stats import spearmanr, pearsonr
from numpy import corrcoef

# Utils
file_path = os.getcwd()  # os.path.dirname(os.path.relpath(__file__))
utils_path = os.path.abspath(os.path.join(file_path, '..', '..', 'utils_py'))
sys.path.append(utils_path)
import utils_all as utils

# Feature importance
pfi_path = os.path.abspath(os.path.join(file_path, '..', 'pfi'))
sys.path.append(pfi_path)
import pfi
import pfi_utils

import warnings
warnings.filterwarnings('ignore')

SEED=0

Using TensorFlow backend.


In [2]:
df_rna, meta = utils.load_lincs1000(dataset='raw', sources=['gdc'])
df_rna[:3]


Dataset: combined_rnaseq_data_lincs1000
df_rna (11081, 943)
meta   (11081, 20)


,Sample,AARS,ABCB6,ABCC5,ABCF1,ABCF3,ABHD4,ABHD6,ABL1,ACAA1,...,ZMIZ1,ZMYM2,ZNF131,ZNF274,ZNF318,ZNF395,ZNF451,ZNF586,ZNF589,ZW10
0,GDC.ADR_ACC.TCGA_OR_A5J1_08081.p,3.76,3.11,1.37,2.94,7.21,3.58,0.61,1.38,2.77,...,0.45,1.53,0.48,1.24,0.77,2.09,1.32,0.23,0.65,0.84
1,GDC.ADR_ACC.TCGA_OR_A5J2_10193.p,5.02,1.65,2.09,3.99,6.57,3.75,2.99,1.84,3.38,...,2.08,2.32,0.84,2.37,1.94,1.77,1.25,0.95,1.20,1.28
2,GDC.ADR_ACC.TCGA_OR_A5J3_09853.p,4.33,2.47,1.77,3.29,5.85,5.42,1.49,2.05,2.98,...,1.46,1.72,0.94,1.61,1.38,2.53,1.16,0.18,0.73,1.20


In [3]:
dd = pd.read_csv('/vol/ml/apartin/projects/feature-importance/apps/data/P1B1.dev.train.lincs', sep='\t')
dd[:3]

,y,AARS,ABCB6,ABCC5,ABCF1,ABCF3,ABHD4,ABHD6,ABL1,ACAA1,...,ZMIZ1,ZMYM2,ZNF131,ZNF274,ZNF318,ZNF395,ZNF451,ZNF586,ZNF589,ZW10
0,3,19.3,15.6,16.3,18.9,17.8,16.9,15.2,18.0,17.3,...,17.1,17.3,16.1,17.0,15.9,16.2,16.2,16.0,16.5,17.2
1,0,20.2,15.6,16.3,18.8,18.1,18.5,18.3,18.9,18.2,...,19.0,16.7,16.2,16.5,16.2,17.8,15.8,14.9,16.7,16.5
2,9,19.6,14.1,16.3,18.8,17.7,18.7,15.5,18.5,17.0,...,17.6,17.8,16.8,15.9,16.0,16.7,15.7,16.1,16.5,17.5
